In [ ]:
import pandas as pd

In [ ]:
# Leer como texto plano
df_raw = pd.read_csv('porcentaje_migrantes_latam.csv', header=None)

# Separar manualmente la única columna por coma
df = df_raw[0].str.split(',', expand=True)

# Tomar la primera fila como encabezado
df.columns = df.iloc[0]
df = df[1:]

# Reiniciar el índice
df.reset_index(drop=True, inplace=True)

# Verificar estructura final
df.head()


,país,1990,1995,2000,2005,2010,2015,2020,2024
0,Belize,16.6,16.4,15.2,14.8,14.5,15.3,15.9,16.5
1,Costa Rica,13.4,10.3,7.9,8.4,8.9,8.5,10.3,12.2
2,El Salvador,0.9,0.7,0.5,0.6,0.7,0.7,0.7,0.7
3,Guatemala,2.9,1.5,0.4,0.4,0.5,0.5,0.5,0.5
4,Honduras,5.4,2.6,0.4,0.4,0.3,0.4,0.4,0.4


In [ ]:
# Transformar el DataFrame de formato ancho a formato largo (tidy)
df_largo = df.melt(id_vars='país', var_name='Año', value_name='Porcentaje')

# Asegurar que los valores sean numéricos (float)
df_largo['Porcentaje'] = pd.to_numeric(df_largo['Porcentaje'], errors='coerce')
df_largo['Año'] = df_largo['Año'].astype(str)  # Mantener años como texto para animaciones

# Verificar estructura final
df_largo.head()

,país,Año,Porcentaje
0,Belize,1990,16.6
1,Costa Rica,1990,13.4
2,El Salvador,1990,0.9
3,Guatemala,1990,2.9
4,Honduras,1990,5.4


In [ ]:
# Instalar Plotly
!pip install plotly

In [ ]:
# Crear un diccionario que asigna una posición numérica a cada país
pais_pos = {pais: i for i, pais in enumerate(df_largo['país'].unique())}

# Crear nueva columna con esa posición
df_largo['x_pos'] = df_largo['país'].map(pais_pos)

In [ ]:
import plotly.express as px

fig = px.scatter(
    df_largo,
    x="x_pos",
    y="Porcentaje",
    size="Porcentaje",
    color="país",
    animation_frame="Año",
    text="país",
    size_max=90,
    title="Evolución del Porcentaje de Población Migrante respecto al Total Nacional (1990–2024)"
)

# Layout visual
fig.update_layout(
    width=1700,
    height=900,
    xaxis=dict(
        showticklabels=False,
        title='',
        tickvals=[],
        ticktext=[]
    ),
    # Animación más lenta
    updatemenus=[{
        "type": "buttons",
        "showactive": False,
        "buttons": [{
            "label": "Play",
            "method": "animate",
            "args": [None, {
                "frame": {"duration": 1000, "redraw": True},  # Aumenta tiempo por frame
                "transition": {"duration": 400},               # Transición más suave
                "fromcurrent": True,
                "mode": "immediate"
            }]
        }, {
            "label": "Pause",
            "method": "animate",
            "args": [[None], {
                "mode": "immediate",
                "frame": {"duration": 0, "redraw": False},
                "transition": {"duration": 0}
            }]
        }]
    }]
)

# Mostrar nombres sobre burbujas
fig.update_traces(textposition='top center')

fig.show()
